In [1]:
import scipy.stats as sts
import math
%matplotlib inline

import os
import sys
import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline
plt.style.use('ggplot')
np.random.seed(1234)

np.set_printoptions(formatter={'all':lambda x: '%.10f' % x})

In [2]:
text_file = open("d://tmp//base_2.txt", "r")
lines = text_file.readlines()
text_file.close()

In [3]:
data = []
for i in lines:
    data.append(float(i))

In [4]:
data = data[0:1040]

In [5]:
k=10

In [6]:
n=len(data)
n

1040

In [7]:
def set_initial_guess(xdata, k):
    np.random.seed(1234)
# set pi mu sigma
    pis = np.random.random(k)
    pis *= 0.9
    pis += 0.1
    pis /= np.sum(pis)
# mu
    mus = np.zeros(k)
# sigma
    sigmas = np.random.random(k)
    sigmas *= 1.5
    sigmas += 0.25
    sigmas *= np.std(xdata)
    return pis, mus, sigmas


In [8]:
pis, mus, sigmas = set_initial_guess(data, k)

In [9]:
pis = [0.05, 0.1, 0.08, 0.1, 0.1, 0.05, 0.05, 0.1, 0.2, 0.17]

In [10]:
mus

array([0.0000000000, 0.0000000000, 0.0000000000, 0.0000000000,
       0.0000000000, 0.0000000000, 0.0000000000, 0.0000000000,
       0.0000000000, 0.0000000000])

In [11]:
sigmas = [1.4, 1.7, 2.2, 2.3, 1.5, 1.9, 1.8, 0.5, 2.5, 2.8]

In [12]:
for i in range (100):
    print('EM iteration {0}'.format(i))
    # E step
    ws = np.zeros((n, k))
    for j in range(k):
        for i in range(n):
            if sigmas[j] != 0:
                ws[i, j] = pis[j] * sts.norm(mus[j], sigmas[j]).pdf(data[i])
            #elif mus[j]-data[i] == 0:
            #    ws[i, j] = pis[j]
            #else:
            #    ws[i, j] = 0
    #S step
    ys = np.zeros((n, k))
    for i in range(n):
        ys[i] = np.random.multinomial(1, ws[i,:]/sum(ws[i,:]), size=1)
    v = ys.sum(0)

    #M step
    ss=[]
    for i in range(k):
        ss.append([])
    for i in range(n):
        for j in range(k):
            if (ys[i,j] == 1):
                ss[j].append(data[i])
    for j in range(k):
        ss[j].sort()
    print(f"v={v}")
    #pi
    pis = v/n
    #mu
    mus = np.zeros(k)
    for j in range(k):
        if (v[j] %2 == 0):
            mus[j]=0.5*(ss[j][int(v[j])//2-1]+ss[j][int(v[j])//2])
        else:
            mus[j]=ss[j][math.floor(v[j]//2)]
    #sigma
    bs=np.zeros(k)
    for j in range(k):
        for i in range(int(v[j])):
            bs[j]+=abs(ss[j][i]-mus[j])
        bs[j] /= v[j]
    
    print(f"bs={bs}")
    
    for j in range(k):
        sigmas[j] = math.sqrt(math.pi/2)*bs[j]
    
    """#SEM
    mus = np.zeros(k)
    for j in range(k):
        for i in range(n):
            if (ys[i,j] == 1):
                mus += data[i]
    mus /= v
    #sigma
    sigmas = np.zeros(k)
    for j in range(k):
        for i in range(n):
            if (ys[i,j] == 1):
                sigmas[j]+= (data[i]-mus[j])**2
    sigmas /= v
    sigmas = np.sqrt(sigmas)
    """
    print(f"p={pis}")
    print(f"u={mus}")
    print(f"s={sigmas}")

EM iteration 0
v=[66.0000000000 125.0000000000 87.0000000000 86.0000000000 117.0000000000
 41.0000000000 34.0000000000 133.0000000000 195.0000000000 156.0000000000]
bs=[0.9069696970 1.0773600000 1.3668965517 1.3951162791 1.0754700855
 1.1714634146 1.0502941176 0.4015037594 1.5879487179 1.8875641026]
p=[0.0634615385 0.1201923077 0.0836538462 0.0826923077 0.1125000000
 0.0394230769 0.0326923077 0.1278846154 0.1875000000 0.1500000000]
u=[0.1100000000 0.0700000000 0.5200000000 -0.1450000000 -0.2100000000
 0.0600000000 -0.0600000000 0.0000000000 0.3100000000 0.1700000000]
s=[1.1367179433288765, 1.3502705189782276, 1.7131507725236705, 1.748518955757136, 1.3479018623795673, 1.468211658908865, 1.3163484659863676, 0.5032103378394567, 1.9901985775371516, 2.3657107748328348]
EM iteration 1
v=[67.0000000000 146.0000000000 106.0000000000 77.0000000000 120.0000000000
 30.0000000000 34.0000000000 133.0000000000 198.0000000000 129.0000000000]
bs=[0.7386567164 1.0804109589 1.5100943396 1.2067532468 1.0

v=[71.0000000000 103.0000000000 128.0000000000 162.0000000000 144.0000000000
 10.0000000000 91.0000000000 72.0000000000 155.0000000000 104.0000000000]
bs=[0.8019718310 0.5943689320 0.9795312500 1.3222839506 0.7789583333
 1.1480000000 1.0603296703 0.4004166667 1.3058064516 3.0032692308]
p=[0.0682692308 0.0990384615 0.1230769231 0.1557692308 0.1384615385
 0.0096153846 0.0875000000 0.0692307692 0.1490384615 0.1000000000]
u=[0.1900000000 -0.1000000000 0.8800000000 -0.8050000000 0.6300000000
 -0.3250000000 0.0600000000 -0.0850000000 -0.2900000000 0.3200000000]
s=[1.005122633503445, 0.7449309853053876, 1.2276603635673229, 1.6572371688540328, 0.9762794915463867, 1.4388046296381942, 1.3289261660392597, 0.5018478691500815, 1.63658568640424, 3.7640397850877245]
EM iteration 12
v=[55.0000000000 102.0000000000 127.0000000000 164.0000000000 134.0000000000
 9.0000000000 101.0000000000 85.0000000000 159.0000000000 104.0000000000]
bs=[0.7914545455 0.6304901961 0.9027559055 1.2952439024 0.7200746269
 1

v=[28.0000000000 124.0000000000 116.0000000000 182.0000000000 126.0000000000
 32.0000000000 48.0000000000 85.0000000000 211.0000000000 88.0000000000]
bs=[0.5739285714 0.4191935484 0.7918103448 1.1564835165 0.6288888889
 1.1306250000 1.0918750000 0.6723529412 1.1600000000 3.1471590909]
p=[0.0269230769 0.1192307692 0.1115384615 0.1750000000 0.1211538462
 0.0307692308 0.0461538462 0.0817307692 0.2028846154 0.0846153846]
u=[1.1150000000 -0.2700000000 1.2250000000 -1.0250000000 0.5950000000
 -0.7800000000 0.5100000000 0.5000000000 -0.1700000000 0.3050000000]
s=[0.7193127923807174, 0.5253812004649976, 0.9923870992450744, 1.4494371407811342, 0.7881953352450813, 1.417028296502337, 1.368462373681362, 0.8426694464421277, 1.4538443992859804, 3.94437898101736]
EM iteration 23
v=[25.0000000000 136.0000000000 113.0000000000 193.0000000000 137.0000000000
 30.0000000000 49.0000000000 80.0000000000 195.0000000000 82.0000000000]
bs=[0.6400000000 0.4093382353 0.7686725664 1.1781347150 0.6119708029
 1.262

v=[14.0000000000 153.0000000000 79.0000000000 211.0000000000 164.0000000000
 35.0000000000 44.0000000000 28.0000000000 258.0000000000 54.0000000000]
bs=[0.3750000000 0.4647712418 0.8927848101 1.2993838863 0.4654268293
 1.2814285714 0.8268181818 0.3614285714 1.1318992248 3.6912962963]
p=[0.0134615385 0.1471153846 0.0759615385 0.2028846154 0.1576923077
 0.0336538462 0.0423076923 0.0269230769 0.2480769231 0.0519230769]
u=[1.5100000000 -0.3300000000 1.6600000000 -0.9000000000 0.6400000000
 -1.5500000000 1.2900000000 0.1700000000 0.0450000000 -0.1800000000]
s=[0.46999280149331246, 0.5825043680033019, 1.11893982411218, 1.628536194444506, 0.5833260250078786, 1.6060325445314343, 1.0362629162622246, 0.4529835382011737, 1.4186253004660683, 4.626353833168494]
EM iteration 34
v=[9.0000000000 132.0000000000 74.0000000000 204.0000000000 166.0000000000
 37.0000000000 42.0000000000 35.0000000000 288.0000000000 53.0000000000]
bs=[0.3211111111 0.4173484848 0.9483783784 1.3560784314 0.4272891566
 1.29702

v=[15.0000000000 134.0000000000 90.0000000000 186.0000000000 118.0000000000
 116.0000000000 33.0000000000 20.0000000000 290.0000000000 38.0000000000]
bs=[0.1440000000 0.4195522388 0.9485555556 1.2909677419 0.3567796610
 1.7115517241 0.5015151515 0.2340000000 0.9316206897 4.1818421053]
p=[0.0144230769 0.1288461538 0.0865384615 0.1788461538 0.1134615385
 0.1115384615 0.0317307692 0.0192307692 0.2788461538 0.0365384615]
u=[1.2400000000 -0.4150000000 1.8100000000 -1.0350000000 0.6000000000
 -0.9300000000 0.7000000000 -0.2150000000 0.2450000000 1.0000000000]
s=[0.18047723577343203, 0.5258307522378911, 1.1888380878069364, 1.6179881217860106, 0.44715699305917395, 2.145111972608788, 0.6285560294718644, 0.29327550813182707, 1.1676133809604428, 5.241161830547528]
EM iteration 45
v=[16.0000000000 152.0000000000 96.0000000000 190.0000000000 111.0000000000
 99.0000000000 31.0000000000 19.0000000000 291.0000000000 35.0000000000]
bs=[0.1393750000 0.4333552632 0.8687500000 1.3710000000 0.3578378378
 1

v=[20.0000000000 147.0000000000 37.0000000000 165.0000000000 98.0000000000
 122.0000000000 30.0000000000 55.0000000000 306.0000000000 60.0000000000]
bs=[0.0510000000 0.3801360544 0.5013513514 1.1325454545 0.2558163265
 1.4195081967 0.5340000000 0.1316363636 0.9993137255 3.9098333333]
p=[0.0192307692 0.1413461538 0.0355769231 0.1586538462 0.0942307692
 0.1173076923 0.0288461538 0.0528846154 0.2942307692 0.0576923077]
u=[1.2250000000 -0.5400000000 1.5500000000 -1.1500000000 0.6350000000
 -0.9100000000 1.4600000000 -0.1100000000 0.6350000000 0.4800000000]
s=[0.0639190210030905, 0.4764298911101372, 0.6283507364108791, 1.4194352293342265, 0.32061821859693457, 1.7790896909860516, 0.669269749326477, 0.16498171553025848, 1.2524540197702834, 4.900249391214053]
EM iteration 56
v=[19.0000000000 151.0000000000 40.0000000000 172.0000000000 95.0000000000
 135.0000000000 31.0000000000 45.0000000000 293.0000000000 59.0000000000]
bs=[0.0384210526 0.3671523179 0.4685000000 1.1730232558 0.2558947368
 1.3

v=[5.0000000000 170.0000000000 42.0000000000 161.0000000000 137.0000000000
 115.0000000000 39.0000000000 27.0000000000 244.0000000000 100.0000000000]
bs=[0.0140000000 0.3837058824 0.4471428571 1.4532298137 0.2984671533
 1.1424347826 0.2564102564 0.1048148148 1.0500819672 2.8117000000]
p=[0.0048076923 0.1634615385 0.0403846154 0.1548076923 0.1317307692
 0.1105769231 0.0375000000 0.0259615385 0.2346153846 0.0961538462]
u=[1.2200000000 -0.6050000000 1.4200000000 -0.4300000000 0.4700000000
 -1.4900000000 1.3600000000 -0.0700000000 0.6400000000 0.4150000000]
s=[0.017546397922417016, 0.48090400692405916, 0.5604104642567879, 1.8213534702342071, 0.3740731027359912, 1.4318296640044388, 0.32136259931166666, 0.13136588920751358, 1.3160825748482678, 3.5239433598899916]
EM iteration 67
v=[2.0000000000 173.0000000000 51.0000000000 169.0000000000 133.0000000000
 121.0000000000 42.0000000000 29.0000000000 234.0000000000 86.0000000000]
bs=[0.0050000000 0.3457803468 0.5035294118 1.4724260355 0.276691729

IndexError: list index out of range

In [95]:
ss.sum(0)/n

array([0.0961538462, 0.1144230769, 0.1096153846, 0.0961538462,
       0.0817307692, 0.0894230769, 0.0865384615, 0.1201923077,
       0.1028846154, 0.1028846154])

In [96]:
n

1040

In [90]:
sigmas

array([1.0874789270, 1.6510121138, 1.6408800424, 1.2063566207,
       1.1770543303, 1.8068250238, 1.0018698205, 0.6163418853,
       2.0169134835, 4.0124983111])

In [64]:
a = ws[:,0]/sum(ws[:,0])

In [85]:
ws[0]

array([0.0000000005, 0.0000006516, 0.0000211332, 0.0000508978,
       0.0000000032, 0.0000016444, 0.0000003665, 0.0000000000,
       0.0001199896, 0.0002829505])

In [62]:
np.random.multinomial(1, a, size=1)

array([[0.0000000000, 0.0000000000, 0.0000000000, 0.0000000000,
        0.0000000000, 0.0000000000, 0.0000000000, 0.0000000000,
        0.0000000000, 1.0000000000]])